In [1]:
import tensorflow as tf
import scipy.io as sio
import numpy as np

In [2]:
IMAGE_HEIGHT = 144
IMAGE_WIDTH = 111
X = tf.placeholder(shape=[None,IMAGE_HEIGHT, IMAGE_WIDTH,1], dtype=tf.float32, name='X-input')
is_training = tf.placeholder(dtype=tf.bool)
sess = tf.InteractiveSession()

In [3]:
def inference(images_placeholder, is_training, depth1, depth2, depth3, dense1_units, dense2_units):

    training_mode = is_training is not None
    print('images_placeholder.shape:', images_placeholder.shape)

    # layer1:bn-conv-relu(depth1)-pool
    with tf.name_scope('conv1'):
        conv = tf.layers.conv2d(
            inputs=images_placeholder,
            filters=depth1,
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu
        )
        print('conv1.shape:', conv.shape)

        pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], strides=2)
        print('pool1.shape:', pool.shape)


    # layer2:bn-conv-relu(depth2)-pool
    with tf.name_scope('conv2'):
        conv = tf.layers.conv2d(
            inputs=pool,
            filters=depth2,
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu
        )
        print('conv2.shape:', conv.shape)
        
#         conv_without_pool = tf.layers.conv2d(
#             inputs=conv,
#             filters=depth2,
#             kernel_size=[3, 3],
#             padding="same",
#             activation=tf.nn.relu
#         )
#         print('conv2_without_pool.shape:', conv_without_pool.shape)

        pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], strides=2)
        print('pool2.shape:', pool.shape)

    with tf.name_scope('conv3'):
        conv = tf.layers.conv2d(
            inputs=pool,
            filters=depth3,
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu
        )
        print('conv3.shape:', conv.shape)

#         conv_without_pool = tf.layers.conv2d(
#             inputs=conv,
#             filters=depth2,
#             kernel_size=[3, 3],
#             padding="same",
#             activation=tf.nn.relu
#         )
#         print('conv3_without_pool.shape:', conv_without_pool.shape)

        pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], strides=2)
        print('pool3.shape:', pool.shape)

In [5]:
def loadEEGData():
    dataPath = 'F:/Deep Learning/eeg/dataset/words/'
    # 4125*144*111
    X = sio.loadmat(dataPath+'X.mat')['X']
    y = sio.loadmat(dataPath+'y.mat')['y']

    X_train = X[0:3500,:,:]
    X_test = X[3500:,:,:]
    y_train = y[0:3500,:]
    y_test = y[3500:,:]

    # labels是从1到125，改成0到57
    y_train = y_train - 1
    y_test = y_test - 1

    X_train = np.reshape(X_train, (-1, 144, 111, 1))
    y_train = np.reshape(y_train, (-1,))
    X_test = np.reshape(X_test, (-1, 144, 111, 1))
    y_test = np.reshape(y_test, (-1,))

    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = loadEEGData()
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (3500, 144, 111, 1)
y_train.shape:  (3500,)
X_test.shape:  (625, 144, 111, 1)
y_test.shape:  (625,)


In [6]:
inference(X, True, 32, 64, 128, 1, 1)

images_placeholder.shape: (?, 144, 111, 1)
conv1.shape: (?, 144, 111, 32)
pool1.shape: (?, 72, 55, 32)
conv2.shape: (?, 72, 55, 64)
pool2.shape: (?, 36, 27, 64)
conv3.shape: (?, 36, 27, 128)
pool3.shape: (?, 18, 13, 128)
